In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.impute import KNNImputer

In [31]:
train = pd.read_csv('train_subset.csv')

In [32]:
test = pd.read_csv('test_subset.csv')

In [33]:
y = pd.read_csv('train_labels.csv')

In [34]:
X_train = train.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

In [35]:
X_test = test.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

In [36]:
# storing missing values for each column in train
val_train = train.isna().sum()

# storing missing values for each column in test
val_test = test.isna().sum()

In [37]:
# creating dataframe to store how many values are missing in each column for both train and test
na_df = pd.DataFrame(columns = train.columns)
na_df.loc['Train'] = val_train
na_df.loc['Test'] = val_test

In [38]:
# identify predictors with large number of missing values 
col_missing = []
for col in na_df.columns:
    if na_df.loc['Train',col]/10000 > .8:
        col_missing.append(col)

In [39]:
# drop predictors with more than 80% missing values
X_train = X_train.drop(col_missing,axis=1)

In [40]:
X_test = X_test.drop(col_missing,axis=1)

In [41]:
imputer = KNNImputer(n_neighbors=2)

In [42]:
X_train = pd.DataFrame(imputer.fit_transform(X_train),columns = X_train.columns)

In [43]:
X_test = pd.DataFrame(imputer.fit_transform(X_test),columns = X_test.columns)

In [44]:
y.target.value_counts()

0    340085
1    118828
Name: target, dtype: int64

In [49]:
X_train.to_csv('X_train.csv')

In [50]:
X_test.to_csv('X_test.csv')